# VENDING MACHINE

## Welcome to my Vending machine!

Please feel free to use it by selecting **Run All** :)

### Variable Declaration and Package Importing

Declaration of global variables and importing of Pandas package

In [ ]:
try:
    import pandas as pd
except (ImportError):
    print("ERROR: Unable to import necessary packages.")

choice = " "
payment = 0
proceed = True
flag = True
sugar = False
sugarStock = 3
mix = False
maintenanceMode = False
purchaseMade = True
productDict = {}
changeBucket = {}
transactionList = []

### Inventory and Change Function

Function definition to allow for the vending machine inventory to be filled as well as the coin holder, both for payment and change.

In [ ]:
def readSystemFiles():
    """ Reads the System Files to start up the Vending Machine"""

    global changeBucket

    fHandleProd = open("Product_Info.txt")
    for line in fHandleProd:
        listDetails = line.strip().split("|")
        productDict[listDetails[0]] = {"Item" : listDetails[2], "Price": float(listDetails[4]), "Stock": int(listDetails[6])}
    fHandleProd.close()

    fHandleCoin = open("Coin_Stock.txt")
    for line in fHandleCoin:
        listDetails = line.strip().split("|")
        changeBucket = {2.00: float(listDetails[1]), 1.00: float(listDetails[3]), 0.50: float(listDetails[5]), 0.20: float(listDetails[7]), 0.10: float(listDetails[9])}
    fHandleCoin.close()

    transactionList.clear()

### Written Functions
Functions that cover specific areas of the programme that do not belong to any particular class are contained in this cell. These functions cover more particular areas of the code requirements.

General Functions:
- sugarMixQuery
- complete

In [ ]:
def sugarMixQuery():
    """ 
    Asks for user input as to whether they would like sugar or not.
    Queries if the user would like the sugar automatically mixed or if they wish to manually mix it.    
    """

    global sugar
    global mix
    global sugarStock
    
    print("Would you like to add sugar?")
    displayOption()
    sugarQuery = input("Please enter your choice: ").upper()

    if sugarQuery == "A":
        sugar = True
        sugarStock = sugarStock - 1
        print("Would you like us to mix the sugar for you?")
        displayOption()

        mixQuery = input("Please enter your choice: ").upper()
        if mixQuery == "A":
            mix = True        
        elif mixQuery == "B":
            mix = False
        else:
            print("Incorrect Input")
            
    elif sugarQuery == "B":
        sugar = False
    else:
        print("Incorrect Input")


def complete():
    """
    Function to ask user if they would like to add any more items or process their current basket.
    """

    global flag
    global proceed
    print("Would you like to add more items?")
    displayOption()
    print()

    selection = input("Please enter your choice: ").upper()
    if selection == "A":
        flag = True
        return flag
    elif selection == "B":
        proceed = True
        flag = False
        return proceed, flag
    else:
        print("Incorrect Input")

### Transaction Class

Class to represent the transactions being carried out as well as the functions that are utilised to carry out the transactions themselves. 

Transaction Functions:
- __init__
- __str__
- selectItem
- addItem
- changeOrder
- rmvFromBasket
- confirmProduct
- coinInput
- coinRefund
- consume

In [ ]:
class transaction:
    """
    Transaction class for representing and recording transactions performed by the user.
    """
    global productDict
    global flag
    global purchaseMade
    
    def __init__(self):
        self.totalPrice = 0
        self.totPayment = 0
        self.basketDict = {"Coffee": 0, "Tea": 0, "Coke":0, "Juice": 0}
        self.coinEntered = {2.00: 0, 1.00: 0, 0.50: 0, 0.20: 0, 0.10: 0}

    
    def __str__(self):
        return "Basket: {}, Total Price: {}".format(self.basketDict, self.totalPrice)


    def selectItem(self):
        choice = input("Please enter your product code: ")
        return choice

    
    def addItem(self, choice, productDict):
        """
        Adds an item to the basket and calculates the new total for that item.
        """
        prod = productDict[choice]["Item"]
        if productDict[choice]["Stock"] <= 0:
            print("This item is unavailable")
        else:
            self.basketDict[prod] = self.basketDict[prod] + 1
            prod = productDict[choice]["Price"]
            self.totalPrice = self.totalPrice + prod


    def changeOrder(self):
        """
        Allows user to remove goods from their current basket.
        """
        print("Is the order correct?")
        print(self.basketDict)
        displayOption()
        confirmation = input("Selection: ").upper()
        if confirmation == "B":
            for productID, productInfo in productDict.items(): 
                print("{}    ".format(productID), end="")
                print(productInfo["Item"] + " $" + str(productInfo["Price"]))
            removal = input("Please enter the item code you would like to remove.").upper()
            if removal == "A1":
                self.rmvFromBasket(removal, productDict)
            elif removal == "A2":
                self.rmvFromBasket(removal, productDict)
            elif removal == "B1":
                self.rmvFromBasket(removal, productDict)
            elif removal == "B2":
                self.rmvFromBasket(removal, productDict)
            else: 
                print("Incorrect Input")

   
    def rmvFromBasket(self, choice, productDict):
        """
        Removes the selected product from the basket of goods and deducts its price from the total.
        """
        prod = productDict[choice]["Item"]
        self.basketDict[prod] = self.basketDict[prod] - 1
        prod = productDict[choice]["Price"]
        self.totalPrice = self.totalPrice - prod


    def confirmProduct(self, selection):
        """
        Function that asks user to confirm their selection choice and then proceeds to payment.
        """
        global sugarStock

        query = True
        print("You have selected: ")
        print(displaySelection(selection))

        print("A    Continue")
        print("B    Cancel Selection")

        confirmation = input("Please select continue if this is correct: ").upper()
        
        if confirmation == "A":
            if selection == "A1":
                if sugarStock > 0:
                    sugarMixQuery()
                else:
                    print("Sorry, sugar is unavailable at the moment.")
            flag = False
        elif confirmation =="B":
            flag = True
            self.rmvFromBasket(selection, productDict)     
        elif confirmation not in ("A", "B"):
            print("Invalid Input.")
                

    def coinInput(self):
        """
        Allow for users to input their coins. Validates that coin is legal tender 
        from list of coins. Shows a total of how much has been paid 
        and how much is still remaining to be paid.
        """
        global purchaseMade
        global changeBucket
        validCoinInput = [2.00, 1.00, 0.50, 0.20, 0.10, 99.0]
        #purchaseMade
        
        while self.totPayment < self.totalPrice:
            print("Please deposit your coin or enter '99' to cancel and return to main.")
            payment = float(input("Enter: "))

            if payment in validCoinInput: # Checks that legal tender is used
                if payment == 99.0:
                    purchaseMade = False
                    return purchaseMade
                self.totPayment += payment
                remaining = self.totalPrice - self.totPayment
                self.coinEntered[payment] = self.coinEntered[payment] + 1

                print("Total Paid: $" + "%.2f" % self.totPayment)

                if self.totalPrice > self.totPayment: # Prompt for further payment
                    print("Please insert: $" + "%.2f" % remaining)
                elif self.totalPrice < self.totPayment: # Calculate using below coinRefund function
                    refund = self.totPayment - self.totalPrice
                    print("Refund due...")
                    print("Dispensing: $" + "%.2f" % refund)
                    self.coinRefund()

            else:
                print("Please only enter legal tender.")
        print("Payment processing...")
        
        for coinKeys in changeBucket: # Adding coins to change bucket
            if coinKeys in self.coinEntered:
                changeBucket[coinKeys] = changeBucket[coinKeys] + self.coinEntered[coinKeys]
            else:
                pass
        print()

    
    def coinRefund(self):
        """
        Function is called if user inputs more coin than is necessary and returns 
        it using smallest number of coins.
        """
        global changeBucket
        changeDue = self.totPayment - self.totalPrice
        amount = float(changeDue)
        
        if changeDue > 0:
            while amount > 0:
                if changeDue % 2.00 == 0:
                    changeBucket[2.00] = changeBucket[2.00] - 1
                    amount = amount - 2.00
                elif changeDue % 1.00 == 0:
                    changeBucket[1.00] = changeBucket[1.00] - 1
                    amount = amount - 1.00
                elif changeDue % 0.50 == 0:
                    changeBucket[0.50] = changeBucket[0.50] - 1
                    amount = amount - 0.50
                elif changeDue % 0.20 == 0:
                    changeBucket[0.20] = changeBucket[0.20] - 1
                    amount = amount - 0.20
                elif changeDue % 0.10 == 0:
                    changeBucket[0.10] = changeBucket[0.10] - 1
                    amount = amount -0.10
        


    def consume(self):
        """ Remove purchased items from inventory list"""

        for akey in self.basketDict.keys():
            if self.basketDict[akey] > 0:
                for productID, productInfo in productDict.items():
                    for key in productInfo:
                        if productInfo[key] == akey:
                            productInfo["Stock"] = productInfo["Stock"] - self.basketDict[akey]
      



### Data Class

Class designed to handle the data output and calculations for the customer so that they may monitor their machine more effectively.

Data Functions:
- __init__
- getData

In [ ]:
class data:
    """ Data class to calculate and present revenue as well as number of particular items sold"""

    

    def __init__(self):
        self.productsSold = {"Coffee": 0, "Tea": 0, "Coke":0, "Juice": 0}

    def getData(self):
        """ Functon to create pandas dataframe from purchased products"""  
        global transactionList
        dataList = []
        revenue = 0

        for transactions in transactionList:
            revenue = transactions.totalPrice + revenue
            values = list(transactions.basketDict.values())
            values.append(transactions.totalPrice)
            dataList.append(values)
            
        df = pd.DataFrame(dataList, columns=["Coffee", "Tea", "Coke", "Juice", "Sale Price"])
        df.index.name = "Transactions"
        df.index = df.index + 1
        print(df)

        print("\nRevenue: {}".format(revenue))
        dfAvg = df[["Sale Price"]].mean()
        print("Mean Sale Price: {}".format(dfAvg.to_string(index=False)))
    

### Display Functions
Functions that display certain components/menus of the vending machince. Essentially make up the UI of the vending machine.

Display Functions:
- displayProducts
- displayVendingHome
- displaySelection
- complete
- displayOption

In [ ]:
def displayProducts():
    """
    Displays Product list and if product is unavailable.
    """
    print("ITEMS AVAILABLE: \n")

    for productID, productInfo in productDict.items(): 
        print("{}    ".format(productID), end="")
        for key in productInfo:
            if productInfo[key] == 0: # Message if out of stock
                message = " CURRENTLY NOT AVAILABLE"
            else:
                message = " "
        print(productInfo["Item"] + " $" + str(productInfo["Price"]) + " " + message)
    print("R    Restart Transaction")
    print("C    Complete Transaction")
    print("X    Cancel Transaction")



# Display Home page of Vending Machine to allow choice of either user or administrator access

def displayVendingHome():
    """
    Function to display main prompt page.
    """
    print("Please select an option from below.")
    print()
    print("A    User Access")
    print("B    Admin Access")
    print("C    Turn Off")


# Displays Product and info

def displaySelection(selection):
    """"
    Displays the selected product and price
    """
    selectedItem = productDict[selection]["Item"]
    selectedItemPrice = productDict[selection]["Price"]

    selectionMsg = "{}    ${}".format(selectedItem, selectedItemPrice)
    return selectionMsg


def displayOption():
    """
    Displays YES or NO prior to requesting user input.
    """
    print("A    Yes")
    print("B    No")           

### Main Program
This is the main body of program which covers the majority of functionality and is responsible for calling the functions as well as handling the class objects.

In [ ]:
try:
    print("Vending Machine Initial bootup...\n...")
    print("Vending Machine Setup Required! Please navigate to Admin Menu and initiate setup.")
    print()

    while proceed == True:
        displayVendingHome()

        choice = input("Selection: ").upper()

        if choice == "A":
            print("User Mode Active...")
            print()

            if productDict == {}:
                print("ERROR: MACHINE NOT CURRENTLY SET UP.\nPlease initiate setup from Admin Menu")
                continue
            elif maintenanceMode == True:
                print("Machine is currently undergoing maintenance and is not available.")
                continue

            objTransaction = transaction() # New transaction is instanciated each time user mode is selected
            print("Welcome!")
            flag = True # Reset flag to True whenever new transaction initiated

            while flag == True:
                purchaseMade = True
                displayProducts()

                #print(productDict)
                
                while True:
                    try:
                        selection = objTransaction.selectItem().upper()
                    except (KeyError):
                        print("Please only enter a valid product code.")
                    else:
                        break

                if selection != "R":
                    if selection == "X":
                        purchaseMade = False
                        flag = False
                    elif selection == "C":
                        purchaseMade = True
                        flag = False
                    elif productDict[selection]["Stock"] == 0:
                        print("This item is not currently avaiable. Please select another option.")
                        continue 

                if selection == "A1":
                    if productDict["A1"]["Stock"] == objTransaction.basketDict["Coffee"]:
                        print("There is no more Coffee available.")
                        continue
                    objTransaction.confirmProduct(selection)
                    objTransaction.addItem(selection, productDict)
                    objTransaction.changeOrder()
                    complete()
                elif selection == "A2":
                    if productDict["A2"]["Stock"] == objTransaction.basketDict["Tea"]:
                        print("There is no more Tea available.")
                        continue
                    objTransaction.confirmProduct(selection)
                    objTransaction.addItem(selection, productDict)
                    objTransaction.changeOrder()
                    complete()
                elif selection == "B1":
                    if productDict["B1"]["Stock"] == objTransaction.basketDict["Coke"]:
                        print("There is no more Coke available.")
                        continue
                    objTransaction.confirmProduct(selection)
                    objTransaction.addItem(selection, productDict)
                    objTransaction.changeOrder()
                    complete()
                elif selection == "B2":
                    if productDict["B2"]["Stock"] == objTransaction.basketDict["Juice"]:
                        print("There is no more Juice available.")
                        continue
                    objTransaction.confirmProduct(selection)
                    objTransaction.addItem(selection, productDict)
                    objTransaction.changeOrder()
                    complete()
                elif selection == "C":
                    flag = False
                elif selection == "R":
                    objTransaction = transaction() # Transaction is restarted and basket emptied
                elif selection == "X":
                    continue

            print(objTransaction)

            if purchaseMade == True:
                objTransaction.coinInput()
                if purchaseMade == True:
                    if objTransaction.basketDict["Coffee"] > 0:
                        print("Boiling water...")
                        if sugar == True:
                            if mix == True:
                                print("Mixing sugar...")
                            else:
                                print("Please take your sugar...")                
                    elif objTransaction.basketDict["Tea"] > 0:
                        print("Boiling water...")
                    
                    print("Dispensing...\nThank you for your purchase!")
                    objTransaction.consume()
                    transactionList.append(objTransaction)            
                else:
                    print("Cancelling...\nPlease take your coin from below.")

            print()

        elif choice == "B":
            while True:
                objData = data()
                print("Admin Access Menu:")
                print("A    Initial Setup and/or Reset Machine")
                print("B    Restock Sugar")
                print("C    Transaction Data")
                print("D    Maintenance Mode")
                print("E    Coin Inventory")
                print("X    Return to Main Menu")

                selection = input("Selection: ").upper()
                if selection == "A":
                    print("Initiaing setup...")
                    print()
                    readSystemFiles()
                    print("Inventory Full.\nChange Bucket Full.\nSetup Complete.")
                elif selection == "B":
                    sugarStock = sugarStock + 3
                    continue            
                elif selection == "C":
                    objData.getData()
                elif selection == "D":
                    maintenanceMode = not maintenanceMode
                    if maintenanceMode == True:
                        print("Entering Maintenance Mode...")
                    else:
                        print("Exiting Maintenance Mode...")
                elif selection == "E":
                    print(changeBucket)
                elif selection == "X":
                    break        
                else:
                    print("Invalid input.")
                    continue
        elif choice == "C":
            proceed = False
            
    print("Goodbye...")
except Exception as ex: # General Exception will cover all exceptions and return type and argument involved
    template = "An exception of type {0} occurred. Arguments:\n{1!r}"
    errorMsg = template.format(type(ex).__name__, ex.args)
    print(errorMsg)